# Проблема поиска собственных значений

### Решенные задачи:
- нахождение min и max собственного значения с помощью степенного метода
- нахождение всех собственных значений с помощью сдвигов в степенном методе

In [1]:
import numpy as np

In [2]:
n = 4

In [3]:
# n random eigenvalues
rnd_eigen_values = np.random.rand(n)[::-1] * 10 - 5
rnd_eigen_values

array([-3.61202859, -0.67626238, -4.27910875,  4.62942143])

In [4]:
# build diagonal matrix
L = np.diag(rnd_eigen_values)
L

array([[-3.61202859,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.67626238,  0.        ,  0.        ],
       [ 0.        ,  0.        , -4.27910875,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  4.62942143]])

In [5]:
# random matrix C
C = np.random.rand(n, n) * 15
assert np.linalg.det(C) != 0, 'det(A) = 0'
C

array([[ 5.77360522, 14.04220212,  2.72714737,  5.41974359],
       [10.01341214,  2.09542634,  7.50338279,  1.03094069],
       [ 5.03022587, 10.48383317,  1.05918499, 14.74461433],
       [ 9.86467122,  6.24899502,  4.09220501,  7.21301075]])

In [6]:
A = np.linalg.inv(C) @ L @ C
A

array([[ 21.839938  ,  19.0525282 ,   8.75860965,  22.97674633],
       [ -3.13516812,  -5.28907275,  -1.47881282,  -1.83121785],
       [-28.53640913, -23.57607699, -11.75957161, -29.04506931],
       [ -4.63157513,  -4.08819098,  -1.39922032,  -8.72927194]])

In [7]:
# check eigenvalues of A
np.linalg.eigvals(A)

array([ 4.62942143, -0.67626238, -4.27910875, -3.61202859])

## Степенной метод

In [8]:
def degree_method(A, n, eps=1e-6, delta=0.01, val='max', verbose=False):
    y0 = np.random.rand(n)
    z = y0 / np.linalg.norm(y0)
    l_cur, l_prev = None, None
    iters = 0
    
    if val == 'min':
        A = np.linalg.inv(A)        
    
    while iters < 2 or abs(l_prev - l_cur) >= eps:
        y = A @ z
            
        l_prev = l_cur
        l_cur = np.mean(z / y if val == 'min' else y / z)
            
        z = y / np.linalg.norm(y)
        iters += 1
    
    if verbose:
        print('Iters:', iters)
    
    return l_cur, z

In [9]:
l, x = degree_method(A, n, val='min', verbose=True)
l, x

Iters: 11


(-0.6762623188747873,
 array([ 0.42659234,  0.04142987, -0.89670235, -0.11057773]))

In [10]:
l, x = degree_method(A, n, val='max', verbose=True)
l, x

Iters: 207


(4.629421904725254,
 array([ 0.60964701, -0.05523942, -0.7827048 , -0.11248258]))

## Обратный степенной метод со сдвигами (переменными и постоянными)

In [11]:
def reverse_degree_method(A, n: int, k: int, eps=1e-6, verbose=False, fixed_shift=False):
    
    def get_for_s(A, n, s0):
        s = s0
        s_prev = None
        y0 = np.random.rand(n)
        z = y0 / np.linalg.norm(y0)
        iters = 0
        
        if fixed_shift:
            mtx = np.linalg.inv(A - s*np.eye(n))
        
        while iters < 1000 and (iters < 2 or abs(s - s_prev) >= eps):
            if fixed_shift:
                y = mtx @ z
            else:
                y = np.linalg.inv(A - s*np.eye(n)) @ z

            s_prev = s
            s = np.mean(z / y)
            
            if not fixed_shift:
                s += s_prev
                
            z = y / np.linalg.norm(y)
            iters += 1

        s = s0 + s if fixed_shift else s
        
        if verbose:
            print(f'Iters for s={s:5f}: iters={iters}')
        
        return s, z
    
    # finding eigenvalues range
    l, x = degree_method(A, n, val='max')
    max_abs_eigval = abs(l)
    
    # start approximations from [-max_abs; max_abs]
    if k < n:
        k = n
    start_approx = np.random.rand(k) * 2*max_abs_eigval - max_abs_eigval
    
    es, ex = [], []
    for s0 in start_approx:
        s, x = get_for_s(A, n, float(s0))
        es.append(s)
        ex.append(x)
    
    return es, ex

In [12]:
from math import log10


# search parameters
eps = 1e-8
fixed_shift = True
round_count = round(-log10(eps) - 3) if fixed_shift else round(-log10(eps) - 2)
k = 20

# search eigenvalues by k approximation values
l, x = reverse_degree_method(A, n, k=k, eps=eps, verbose=False, fixed_shift=fixed_shift)

# rounding up results (eigenvalues) and removing repetitions
s_v = {round(s, round_count):  tuple(v) for s, v in zip(l, x)}

# print results
for i, s in enumerate(s_v):
    print(f's[{i+1}]={s}; v[{i+1}]={s_v[s]}')

s[1]=-3.61203; v[1]=(-0.1396106019775848, 0.8275567280278153, 0.029802643577752298, -0.5429277522351665)
s[2]=-4.27911; v[2]=(0.559260899222389, 0.07170929391915626, -0.7079289683371663, -0.42534879752463917)
s[3]=4.62942; v[3]=(0.6096470061131181, -0.055239420883992446, -0.7827048013530742, -0.11248256867869742)
s[4]=-0.67626; v[4]=(0.42659233791584983, 0.04142987496461862, -0.896702351629668, -0.1105777340775642)
